In [141]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [142]:
cols = ['engine_id', 'cycle'] + \
       [f'op_{i}' for i in range(1,4)] + \
       [f'sensor_{i}' for i in range(1,22)]

train = pd.read_csv("../data/train_FD004.txt", sep="\s+", header=None)
train.columns = cols

test = pd.read_csv("../data/test_FD004.txt", sep="\s+", header=None)
test.columns = cols

rul = pd.read_csv("../data/RUL_FD004.txt", header=None)

<h3>CREATE RUL FOR TRAIN</h3>

In [143]:
max_cycle = train.groupby('engine_id')['cycle'].max().reset_index()
max_cycle.columns = ['engine_id','max_cycle']

train = train.merge(max_cycle, on='engine_id')
train['RUL'] = train['max_cycle'] - train['cycle']
train.drop(columns=['max_cycle'], inplace=True)

# Cap RUL
RUL_CAP = 130
train['RUL'] = train['RUL'].clip(upper=RUL_CAP)

<h3>REMOVE CONSTANT SENSORS</h3>

In [144]:
sensor_cols = [col for col in train.columns if "sensor" in col]

nunique = train[sensor_cols].nunique()
variance = train[sensor_cols].var()

drop_cols = list(set(
    nunique[nunique == 1].index.tolist() +
    variance[variance < 1e-3].index.tolist()
))

train.drop(columns=drop_cols, inplace=True)
test.drop(columns=drop_cols, inplace=True)

<h3>Feature Scaling</h3>

In [145]:
features = [col for col in train.columns if col not in ['engine_id','cycle','RUL']]

scaler = MinMaxScaler()

train[features] = scaler.fit_transform(train[features])
test[features] = scaler.transform(test[features])

<h3>Create Sliding Windows</h3>

In [146]:
def create_sequences(df, seq_length):
    X, y = [], []

    for engine in df['engine_id'].unique():
        engine_data = df[df['engine_id'] == engine]
        engine_data = engine_data.sort_values('cycle')

        data = engine_data[features].values
        rul = engine_data['RUL'].values

        for i in range(len(data) - seq_length):
            X.append(data[i:i+seq_length])
            y.append(rul[i+seq_length])

    return np.array(X), np.array(y)

SEQ_LENGTH = 50

X_train_seq, y_train_seq = create_sequences(train, SEQ_LENGTH)

print("Sequence shape:", X_train_seq.shape)

Sequence shape: (48799, 50, 23)


<h3>Train/Validation Split</h3>

In [147]:
engine_ids = train['engine_id'].unique()
train_ids, val_ids = train_test_split(engine_ids, test_size=0.2, random_state=42)

train_df = train[train['engine_id'].isin(train_ids)]
val_df = train[train['engine_id'].isin(val_ids)]

X_train_seq, y_train_seq = create_sequences(train_df, SEQ_LENGTH)
X_val_seq, y_val_seq = create_sequences(val_df, SEQ_LENGTH)

# Scale target
y_train_seq = y_train_seq / RUL_CAP
y_val_seq = y_val_seq / RUL_CAP

<h3>Build LSTM Model</h3>

In [148]:
model = Sequential()

model.add(LSTM(128, return_sequences=True, 
               input_shape=(SEQ_LENGTH, len(features))))
model.add(Dropout(0.3))

model.add(LSTM(64))
model.add(Dropout(0.3))

model.add(Dense(32, activation='relu'))
model.add(Dense(1))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='mse'
)

<h3>Train</h3>

In [149]:
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(
    X_train_seq, y_train_seq,
    validation_data=(X_val_seq, y_val_seq),
    epochs=30,
    batch_size=128,
    callbacks=[early_stop]
)

Epoch 1/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 81s 249ms/step - loss: 0.1125 - val_loss: 0.0642
Epoch 2/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 76s 246ms/step - loss: 0.0538 - val_loss: 0.0476
Epoch 3/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 76s 245ms/step - loss: 0.0397 - val_loss: 0.0494
Epoch 4/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 77s 250ms/step - loss: 0.0335 - val_loss: 0.0361
Epoch 5/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 75s 245ms/step - loss: 0.0273 - val_loss: 0.0342
Epoch 6/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 76s 247ms/step - loss: 0.0250 - val_loss: 0.0341
Epoch 7/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 76s 248ms/step - loss: 0.0228 - val_loss: 0.0258
Epoch 8/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 95s 307ms/step - loss: 0.0216 - val_loss: 0.0300
Epoch 9/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 85s 275ms/step - loss: 0.0222 - val_loss: 0.0307
Epoch 10/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 77s 251ms/step - loss: 0.0219 - val_loss: 0.0324
Epoch 11/30
308/308 ━━━━━━━━━━━━━━━━━━━━ 76s 248ms/step - loss: 0.0199 - val_loss: 0.0273
Epoch 12/30
308/308

<h3>Evaluate on Test Set</h3>

In [151]:
X_test_seq = []
y_test = []

for i, engine in enumerate(test['engine_id'].unique()):
    
    engine_data = test[test['engine_id'] == engine]
    engine_data = engine_data.sort_values('cycle')
    
    data = engine_data[features].values
    
    # If engine has less than SEQ_LENGTH cycles
    if len(data) < SEQ_LENGTH:
        padding = np.zeros((SEQ_LENGTH - len(data), data.shape[1]))
        data = np.vstack((padding, data))
    
    last_window = data[-SEQ_LENGTH:]
    
    X_test_seq.append(last_window)
    y_test.append(rul.iloc[i, 0])

X_test_seq = np.array(X_test_seq)
y_test = np.array(y_test)

pred_test = model.predict(X_test_seq)
pred_test = pred_test.flatten() * RUL_CAP

rmse = np.sqrt(mean_squared_error(y_test, pred_test))
print("FINAL TEST RMSE:", rmse)

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
FINAL TEST RMSE: 29.641874891280285


<h3>APR MODEL</h3>

In [152]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Healthy data (normal behavior)
healthy_data = train[train['RUL'] > 100][features].values

input_dim = healthy_data.shape[1]

input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation="relu")(input_layer)
encoded = Dense(32, activation="relu")(encoded)
decoded = Dense(64, activation="relu")(encoded)
output_layer = Dense(input_dim, activation="sigmoid")(decoded)

apr_model = Model(input_layer, output_layer)
apr_model.compile(optimizer=Adam(0.001), loss='mse')

apr_model.fit(
    healthy_data,
    healthy_data,
    epochs=30,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

Epoch 1/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0314 - val_loss: 0.0029
Epoch 2/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0017 - val_loss: 0.0010
Epoch 3/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.6979e-04 - val_loss: 4.1993e-04
Epoch 4/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 3.7469e-04 - val_loss: 3.1074e-04
Epoch 5/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2.6702e-04 - val_loss: 1.9781e-04
Epoch 6/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 1.5107e-04 - val_loss: 1.0542e-04
Epoch 7/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9.0284e-05 - val_loss: 7.4287e-05
Epoch 8/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 7.3407e-05 - val_loss: 6.4858e-05
Epoch 9/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.7554e-05 - val_loss: 6.2014e-05
Epoch 10/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.4541e-05 - val_loss: 5.8545e-05
Epoch 11/30
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 6.2873e-05 - v

<h3>Reconstruction Error & Threshold</h3>

In [153]:
reconstructions = apr_model.predict(healthy_data)

mse = np.mean(np.square(healthy_data - reconstructions), axis=1)

threshold = np.mean(mse) + 3*np.std(mse)

print("APR Threshold:", threshold)

1129/1129 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
APR Threshold: 0.0001300935959411916


<h3>Detect Alerts (On Test Last Cycle)</h3>

In [154]:
test_features = test_last[features].values

recon_test = apr_model.predict(test_features)

mse_test = np.mean(np.square(test_features - recon_test), axis=1)

test_last['anomaly_score'] = mse_test
test_last['alert'] = test_last['anomaly_score'] > threshold

test_last[['engine_id','anomaly_score','alert']].head()

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


,engine_id,anomaly_score,alert
0,1,0.000091,False
1,2,0.000105,False
2,3,0.000052,False
3,4,0.000026,False
4,5,0.000059,False


<h3>Health Index</h3>

In [155]:


# RUL-based health
test_last['predicted_RUL'] = pred_test
test_last['rul_health'] = test_last['predicted_RUL'] / RUL_CAP

# APR-based health
test_last['apr_health'] = 1 - (test_last['anomaly_score'] / threshold)
test_last['apr_health'] = test_last['apr_health'].clip(lower=0)

# Combined Health Index
test_last['health_index'] = 0.7 * test_last['rul_health'] + \
                            0.3 * test_last['apr_health']

test_last[['engine_id','health_index']].head()

,engine_id,health_index
0,1,0.234362
1,2,0.447237
2,3,0.762592
3,4,0.729844
4,5,0.736994


In [156]:
def severity(score, threshold):
    if score > 2 * threshold:
        return "CRITICAL"
    elif score > threshold:
        return "WARNING"
    else:
        return "NORMAL"

test_last['severity'] = test_last['anomaly_score'].apply(
    lambda x: severity(x, threshold)
)

test_last[['engine_id','severity']].head()

,engine_id,severity
0,1,NORMAL
1,2,NORMAL
2,3,NORMAL
3,4,NORMAL
4,5,NORMAL


In [157]:
import joblib
import json

# Save LSTM RUL model
model.save("lstm_rul_model.h5")

# Save APR model
apr_model.save("apr_autoencoder.h5")

# Save scaler
joblib.dump(scaler, "scaler.pkl")

# Save threshold
with open("apr_threshold.json", "w") as f:
    json.dump({"threshold": float(threshold)}, f)

print("All models saved successfully.")

All models saved successfully.
